#Charger un fichier Excel, afficher les feuilles et convertir une feuille en CSV

Ce script :
1. Demande à l'utilisateur de téléverser un fichier Excel via Google Colab.
2. Récupère le nom du fichier téléchargé et l'affiche.
3. Charge le fichier Excel et liste les noms des feuilles disponibles.
4. Charge une feuille spécifique (par défaut : `fwd_month`) dans un DataFrame Pandas.
   - Si la feuille spécifiée n'existe pas, une erreur est levée.
5. Sauvegarde le contenu de la feuille sélectionnée dans un fichier CSV portant le nom de la feuille (par exemple : `fwd_month.csv`).

**Utilisation** : Ce script importe des données depuis un fichier Excel et les convertir en un fichier CSV exploitable dans Pandas ou d'autres outils.


In [2]:
# Import necessary libraries
import pandas as pd
from google.colab import files

# Step 1: Upload the Excel file
print("Please upload your Excel file:")
uploaded = files.upload()  # This will prompt you to upload a file

# Step 2: Retrieve the filename from the uploaded file
filename = list(uploaded.keys())[0]  # Get the uploaded file's name
print(f"File '{filename}' uploaded successfully!")

# Step 3: Load the Excel file and list available sheets
excel_file = pd.ExcelFile(filename)
print("Available sheets:")
print(excel_file.sheet_names)

# Step 4: Retrieve a specific sheet (example: 'fwd_month') and convert it to a DataFrame
sheet_name = "fwd_month"  # Replace with your desired sheet name
if sheet_name in excel_file.sheet_names:
    df = excel_file.parse(sheet_name)
    print(f"Sheet '{sheet_name}' loaded successfully!")
else:
    raise ValueError(f"Sheet '{sheet_name}' not found in the Excel file!")

# Step 5: Save the DataFrame to a CSV file
csv_filename = f"{sheet_name}.csv"
df.to_csv(csv_filename, index=False)
print(f"Sheet '{sheet_name}' has been saved as '{csv_filename}'")


Please upload your Excel file:


Saving Projet_Finance_Internationale_CODE (3) ouss.xls to Projet_Finance_Internationale_CODE (3) ouss.xls
File 'Projet_Finance_Internationale_CODE (3) ouss.xls' uploaded successfully!
Available sheets:
['Rendement_Pf', 'Portefeuille', 'prime forward', 'Forward log vs USD', 'Forward weekly versus USD', 'Forward versus USD', 'Forward log weekly vs USD', 'fwd_month', 'SPOT mensuel log vs USD', 'Spot_1er_j_log', 'SPOT versus USD', 'spot_daily']
Sheet 'fwd_month' loaded successfully!
Sheet 'fwd_month' has been saved as 'fwd_month.csv'


# Afficher un fichier CSV avec des styles pour les colonnes, les données et les valeurs manquantes

Ce script :
1. Charge un fichier CSV spécifié (`fwd_month.csv`) dans un DataFrame Pandas.
2. Affiche les noms des colonnes dans une table stylisée avec un titre descriptif ("List of Columns").
3. Montre les premières lignes du DataFrame dans une table stylisée avec un titre descriptif ("Dataset Preview").
4. Affiche un tableau des valeurs manquantes (N/A) par colonne, avec un dégradé de couleurs (`coolwarm`) pour faciliter la lecture.

**Utilisation** :
- Ce script est utile pour inspecter un fichier CSV, vérifier les colonnes, explorer les premières lignes du jeu de données et identifier les colonnes contenant des valeurs manquantes.


In [3]:
# Import necessary libraries
import pandas as pd
from IPython.display import display, HTML

# Load the CSV file
csv_filename = "fwd_month.csv"  # Replace with the name of your CSV file
df = pd.read_csv(csv_filename)

# Step 1: Display all column names in a styled table
print("Column Names:")
styled_columns = pd.DataFrame(df.columns, columns=["Column Names"])
display(styled_columns.style.set_caption("List of Columns").set_table_styles(
    [{'selector': 'caption', 'props': [('font-size', '16px'), ('text-align', 'left')]}]
))

# Step 2: Display the first few rows of the dataset in a styled format
print("\nPreview of the Dataset:")
display(df.head().style.set_caption("Dataset Preview").set_table_styles(
    [{'selector': 'caption', 'props': [('font-size', '16px'), ('text-align', 'left')]}]
))

# Step 3: Display missing values (N/A counts) in a styled table
print("\nMissing Values (N/A counts) per Column:")
missing_values = pd.DataFrame(df.isnull().sum(), columns=["Missing Values"])
display(missing_values.style.set_caption("Missing Values Per Column").set_table_styles(
    [{'selector': 'caption', 'props': [('font-size', '16px'), ('text-align', 'left')]}]
).background_gradient(cmap='coolwarm', axis=0))


Column Names:


,Column Names
0,Unnamed: 0
1,NOK TO US $ 1M FWD (LSEG DS) - BID SPOT
2,NOK TO US $ 1M FWD (LSEG DS) - EXCHANGE RATE
3,NOK TO US $ 1M FWD (LSEG DS) - SPOT OFFERED
4,NOK TO US $ 1W FWD (LSEG DS) - BID SPOT
5,NOK TO US $ 1W FWD (LSEG DS) - EXCHANGE RATE
6,NOK TO US $ 1W FWD (LSEG DS) - SPOT OFFERED
7,PHP TO USD 1M FWD OR (WMR) - BID SPOT
8,PHP TO USD 1M FWD OR (WMR) - EXCHANGE RATE
9,PHP TO USD 1M FWD OR (WMR) - SPOT OFFERED



Preview of the Dataset:


,Unnamed: 0,NOK TO US $ 1M FWD (LSEG DS) - BID SPOT,NOK TO US $ 1M FWD (LSEG DS) - EXCHANGE RATE,NOK TO US $ 1M FWD (LSEG DS) - SPOT OFFERED,NOK TO US $ 1W FWD (LSEG DS) - BID SPOT,NOK TO US $ 1W FWD (LSEG DS) - EXCHANGE RATE,NOK TO US $ 1W FWD (LSEG DS) - SPOT OFFERED,PHP TO USD 1M FWD OR (WMR) - BID SPOT,PHP TO USD 1M FWD OR (WMR) - EXCHANGE RATE,PHP TO USD 1M FWD OR (WMR) - SPOT OFFERED,PHP TO GBP SW FWD OR (WMR) - BID SPOT,PHP TO GBP SW FWD OR (WMR) - EXCHANGE RATE,PHP TO GBP SW FWD OR (WMR) - SPOT OFFERED,PLN TO US $ 1M FWD (LSEG DS) - BID SPOT,PLN TO US $ 1M FWD (LSEG DS) - EXCHANGE RATE,PLN TO US $ 1M FWD (LSEG DS) - SPOT OFFERED,PLN TO US $ 1W FWD (LSEG DS) - BID SPOT,PLN TO US $ 1W FWD (LSEG DS) - EXCHANGE RATE,PLN TO US $ 1W FWD (LSEG DS) - SPOT OFFERED,RUB TO USD 1M FWD OR (WMR) - BID SPOT,RUB TO USD 1M FWD OR (WMR) - EXCHANGE RATE,RUB TO USD 1M FWD OR (WMR) - SPOT OFFERED,RUB TO USD SW FWD OR (WMR) - BID SPOT,RUB TO USD SW FWD OR (WMR) - EXCHANGE RATE,RUB TO USD SW FWD OR (WMR) - SPOT OFFERED,SINGAPORE $ TO US $ 1M FWD (BBI) - BID SPOT,SINGAPORE $ TO US $ 1M FWD (BBI) - EXCHANGE RATE,SINGAPORE $ TO US $ 1M FWD (BBI) - SPOT OFFERED,SINGAPORE $ TO US $ 1W FWD (LSEG DS) - BID SPOT,SINGAPORE $ TO US $ 1W FWD (LSEG DS) - EXCHANGE RATE,SINGAPORE $ TO US $ 1W FWD (LSEG DS) - SPOT OFFERED,ZAR TO USD 1M FWD (LSEG DS) - BID SPOT,ZAR TO USD 1M FWD (LSEG DS) - EXCHANGE RATE,ZAR TO USD 1M FWD (LSEG DS) - SPOT OFFERED,ZAR TO USD 1W FWD (LSEG DS) - BID SPOT,ZAR TO USD 1W FWD (LSEG DS) - EXCHANGE RATE,ZAR TO USD 1W FWD (LSEG DS) - SPOT OFFERED,SEK TO US $ 1M FWD (LSEG DS) - BID SPOT,SEK TO US $ 1M FWD (LSEG DS) - EXCHANGE RATE,SEK TO US $ 1M FWD (LSEG DS) - SPOT OFFERED,SEK TO US $ 1W FWD (LSEG DS) - BID SPOT,SEK TO US $ 1W FWD (LSEG DS) - EXCHANGE RATE,SEK TO US $ 1W FWD (LSEG DS) - SPOT OFFERED,SWISS FRANC TO US $ 1M FWD (LSEG DS) - BID SPOT,SWISS FRANC TO US $ 1M FWD (LSEG DS) - EXCHANGE RATE,SWISS FRANC TO US $ 1M FWD (LSEG DS) - SPOT OFFERED,SWISS FRANC TO US $ 1W FWD (LSEG DS) - BID SPOT,SWISS FRANC TO US $ 1W FWD (LSEG DS) - EXCHANGE RATE,SWISS FRANC TO US $ 1W FWD (LSEG DS) - SPOT OFFERED,THB TO USD 1M FWD OR (WMR) - BID SPOT,THB TO USD 1M FWD OR (WMR) - EXCHANGE RATE,THB TO USD 1M FWD OR (WMR) - SPOT OFFERED,US $ TO GBP 1M FWD (LSEG DS) - BID SPOT,US $ TO GBP 1M FWD (LSEG DS) - EXCHANGE RATE,US $ TO GBP 1M FWD (LSEG DS) - SPOT OFFERED,US $ TO GBP 1W FWD (LSEG DS) - BID SPOT,US $ TO GBP 1W FWD (LSEG DS) - EXCHANGE RATE,US $ TO GBP 1W FWD (LSEG DS) - SPOT OFFERED,Unnamed: 58,Unnamed: 59,Unnamed: 60,Unnamed: 61,US $ TO AUD 1M FWD (LSEG DS) - EXCHANGE RATE,US $ TO AUD 1M FWD (LSEG DS) - SPOT OFFERED,BRL TO USD 1M FWD OFF (LSEG DS) - BID SPOT,BRL TO USD 1M FWD OFF (LSEG DS) - EXCHANGE RATE,BRL TO USD 1M FWD OFF (LSEG DS) - SPOT OFFERED,BRL TO USD SW FWD OR (WMR) - BID SPOT,BRL TO USD SW FWD OR (WMR) - EXCHANGE RATE,BRL TO USD SW FWD OR (WMR) - SPOT OFFERED,BGN TO EUR 1M FWD OR (WMR) - BID SPOT,BGN TO EUR 1M FWD OR (WMR) - EXCHANGE RATE,BGN TO EUR 1M FWD OR (WMR) - SPOT OFFERED,BGN TO EUR SW FWD OR (WMR) - BID SPOT,BGN TO EUR SW FWD OR (WMR) - EXCHANGE RATE,BGN TO EUR SW FWD OR (WMR) - SPOT OFFERED,CANADIAN $ TO US $ 1M FWD (BBI) - BID SPOT,CANADIAN $ TO US $ 1M FWD (BBI) - EXCHANGE RATE,CANADIAN $ TO US $ 1M FWD (BBI) - SPOT OFFERED,CLP TO EUR 1M FWD OR (WMR) - BID SPOT,CLP TO EUR 1M FWD OR (WMR) - EXCHANGE RATE,CLP TO EUR 1M FWD OR (WMR) - SPOT OFFERED,CLP TO EUR SW FWD OR (WMR) - BID SPOT,CLP TO EUR SW FWD OR (WMR) - EXCHANGE RATE,CLP TO EUR SW FWD OR (WMR) - SPOT OFFERED,HRK TO USD 1M FWD OR (WMR) - BID SPOT,HRK TO USD 1M FWD OR (WMR) - EXCHANGE RATE,HRK TO USD 1M FWD OR (WMR) - SPOT OFFERED,HRK TO USD SW FWD OR (WMR) - BID SPOT,HRK TO USD SW FWD OR (WMR) - EXCHANGE RATE,HRK TO USD SW FWD OR (WMR) - SPOT OFFERED,CZK TO US $ 1M FWD (LSEG DS) - BID SPOT,CZK TO US $ 1M FWD (LSEG DS) - EXCHANGE RATE,CZK TO US $ 1M FWD (LSEG DS) - SPOT OFFERED,CZK TO US $ 1W FWD (LSEG DS) - BID SPOT,CZK TO US $ 1W FWD (LSEG DS) - EXCHANGE RATE,C


Missing Values (N/A counts) per Column:


,Missing Values
Unnamed: 0,1
NOK TO US $ 1M FWD (LSEG DS) - BID SPOT,1
NOK TO US $ 1M FWD (LSEG DS) - EXCHANGE RATE,1
NOK TO US $ 1M FWD (LSEG DS) - SPOT OFFERED,1
NOK TO US $ 1W FWD (LSEG DS) - BID SPOT,1
NOK TO US $ 1W FWD (LSEG DS) - EXCHANGE RATE,1
NOK TO US $ 1W FWD (LSEG DS) - SPOT OFFERED,1
PHP TO USD 1M FWD OR (WMR) - BID SPOT,1
PHP TO USD 1M FWD OR (WMR) - EXCHANGE RATE,1
PHP TO USD 1M FWD OR (WMR) - SPOT OFFERED,1


# Corriger les duplications et filtrer les colonnes spécifiques dans un fichier CSV

Ce script :
1. Charge un fichier CSV (`fwd_month.csv`) dans un DataFrame Pandas.
2. Identifie une colonne de départ spécifique (`US $ TO AUD 1M FWD (LSEG DS) - EXCHANGE RATE`) où une duplication commence.
3. À partir de cette colonne, supprime la première ligne et décale toutes les lignes vers le haut pour ces colonnes, tout en conservant intacte la première colonne (par exemple, une colonne de dates).
4. Supprime la dernière ligne des colonnes affectées, car elle devient vide après le décalage.
5. Filtre les colonnes qui contiennent "1M" et "EXCHANGE RATE" dans leurs noms.
6. Inclut la première colonne d'origine (souvent une colonne clé) dans le DataFrame filtré.
7. Sauvegarde le DataFrame filtré dans un nouveau fichier CSV (`filtered_fwd_month.csv`).
8. Affiche la liste des colonnes filtrées ainsi qu'un aperçu des premières lignes du DataFrame filtré.

**Utilisation** :
- Ce script est idéal pour nettoyer des jeux de données où des duplications existent à partir de colonnes spécifiques et pour extraire uniquement les colonnes pertinentes pour une analyse ultérieure.


In [8]:
# Import necessary library
import pandas as pd

# Load the CSV file
csv_filename = "fwd_month.csv"  # Replace with the name of your CSV file
df = pd.read_csv(csv_filename)

# Locate the starting column where the duplication begins
start_column = "US $ TO AUD 1M FWD (LSEG DS) - EXCHANGE RATE"

# Step 1: Get the index of the starting column
if start_column in df.columns:
    start_col_idx = df.columns.get_loc(start_column)  # Get the index of the starting column

    # Step 2: For all columns starting from this index (excluding the first column), remove the first row and shift rows up
    df.iloc[:, start_col_idx:] = df.iloc[:, start_col_idx:].shift(-1, axis=0)

    # Step 3: Drop the last row, as it becomes NaN after the shift (keep the first column unchanged)
    df.iloc[-1, start_col_idx:] = None  # Keep the first column intact while dropping NaNs

# Step 4: Filter columns based on the condition
filtered_columns = [col for col in df.columns if "1M" in col and "EXCHANGE RATE" in col]

# Step 5: Add the first column to the filtered DataFrame
filtered_columns = [df.columns[0]] + filtered_columns  # Ensure the first column is included
filtered_df = df[filtered_columns]

# Optional: Save the filtered DataFrame to a new CSV file
filtered_csv_filename = "filtered_fwd_month.csv"
filtered_df.to_csv(filtered_csv_filename, index=False)
print(f"Filtered data saved to '{filtered_csv_filename}'")

# Step 6: Display the filtered DataFrame
print("\nFiltered Columns:")
print(filtered_columns)

print("\nPreview of Filtered DataFrame:")
print(filtered_df.head())


Filtered data saved to 'filtered_fwd_month.csv'

Filtered Columns:
['Unnamed: 0', 'NOK TO US $ 1M FWD (LSEG DS) - EXCHANGE RATE', 'PHP TO USD 1M FWD OR (WMR) - EXCHANGE RATE', 'PLN TO US $ 1M FWD (LSEG DS) - EXCHANGE RATE', 'RUB TO USD 1M FWD OR (WMR) - EXCHANGE RATE', 'SINGAPORE $ TO US $ 1M FWD (BBI) - EXCHANGE RATE', 'ZAR TO USD 1M FWD (LSEG DS) - EXCHANGE RATE', 'SEK TO US $ 1M FWD (LSEG DS) - EXCHANGE RATE', 'SWISS FRANC TO US $ 1M FWD (LSEG DS) - EXCHANGE RATE', 'THB TO USD 1M FWD OR (WMR) - EXCHANGE RATE', 'US $ TO GBP 1M FWD (LSEG DS) - EXCHANGE RATE', 'US $ TO AUD 1M FWD (LSEG DS) - EXCHANGE RATE', 'BRL TO USD 1M FWD OFF (LSEG DS) - EXCHANGE RATE', 'BGN TO EUR 1M FWD OR (WMR) - EXCHANGE RATE', 'CANADIAN $ TO US $ 1M FWD (BBI) - EXCHANGE RATE', 'CLP TO EUR 1M FWD OR (WMR) - EXCHANGE RATE', 'HRK TO USD 1M FWD OR (WMR) - EXCHANGE RATE', 'CZK TO US $ 1M FWD (LSEG DS) - EXCHANGE RATE', 'US $ TO EURO 1M FWD (LSEG DS) - EXCHANGE RATE', 'HUF TO USD 1M FWD OR (WMR) - EXCHANGE RATE', 'I

# Afficher un DataFrame filtré de manière stylisée

Ce script :
1. Charge un fichier CSV filtré (`filtered_fwd_month.csv`) dans un DataFrame Pandas.
2. Affiche le DataFrame de manière stylisée à l'aide de Pandas Styler, avec :
   - Un titre descriptif : "Filtered DataFrame: Columns with '1M' and 'EXCHANGE RATE'".
   - Une mise en forme visuelle pour améliorer la lisibilité, incluant un dégradé de couleurs (`coolwarm`) pour mettre en évidence les variations dans les données.

**Utilisation** :
- Ce script est utilisé pour afficher et explorer un DataFrame contenant des colonnes filtrées avec des critères spécifiques (par exemple, "1M" et "EXCHANGE RATE") tout en offrant une présentation visuelle améliorée pour les données.


In [9]:
# Import necessary libraries
import pandas as pd
from IPython.display import display

# Load the filtered CSV file
filtered_csv_filename = "filtered_fwd_month.csv"  # Replace with your filtered CSV filename
filtered_df = pd.read_csv(filtered_csv_filename)

# Display the filtered DataFrame in a styled manner
print("Displaying Filtered DataFrame:")
display(filtered_df.style.set_caption("Filtered DataFrame: Columns with '1M' and 'EXCHANGE RATE'")
        .set_table_styles([{'selector': 'caption', 'props': [('font-size', '16px'), ('text-align', 'left')]}])
        .background_gradient(cmap="coolwarm", axis=None))  # Add a gradient for better readability


Displaying Filtered DataFrame:


,Unnamed: 0,NOK TO US $ 1M FWD (LSEG DS) - EXCHANGE RATE,PHP TO USD 1M FWD OR (WMR) - EXCHANGE RATE,PLN TO US $ 1M FWD (LSEG DS) - EXCHANGE RATE,RUB TO USD 1M FWD OR (WMR) - EXCHANGE RATE,SINGAPORE $ TO US $ 1M FWD (BBI) - EXCHANGE RATE,ZAR TO USD 1M FWD (LSEG DS) - EXCHANGE RATE,SEK TO US $ 1M FWD (LSEG DS) - EXCHANGE RATE,SWISS FRANC TO US $ 1M FWD (LSEG DS) - EXCHANGE RATE,THB TO USD 1M FWD OR (WMR) - EXCHANGE RATE,US $ TO GBP 1M FWD (LSEG DS) - EXCHANGE RATE,US $ TO AUD 1M FWD (LSEG DS) - EXCHANGE RATE,BRL TO USD 1M FWD OFF (LSEG DS) - EXCHANGE RATE,BGN TO EUR 1M FWD OR (WMR) - EXCHANGE RATE,CANADIAN $ TO US $ 1M FWD (BBI) - EXCHANGE RATE,CLP TO EUR 1M FWD OR (WMR) - EXCHANGE RATE,HRK TO USD 1M FWD OR (WMR) - EXCHANGE RATE,CZK TO US $ 1M FWD (LSEG DS) - EXCHANGE RATE,US $ TO EURO 1M FWD (LSEG DS) - EXCHANGE RATE,HUF TO USD 1M FWD OR (WMR) - EXCHANGE RATE,INR TO USD 1M FWD OR (WMR) - EXCHANGE RATE,IDR TO USD 1M FWD OR (WMR) - EXCHANGE RATE,ILS TO USD 1M FWD OR (WMR) - EXCHANGE RATE,JPY TO US $ 1M FWD (LSEG DS) - EXCHANGE RATE,MXN TO USD 1M FWD OR (WMR) - EXCHANGE RATE,US $ TO NZD 1M FWD (LSEG DS) - EXCHANGE RATE
0,Code,TDNOK1M(ER),USPHP1F(ER),TDPLN1M(ER),USRUB1F(ER),BBSGD1F(ER),TDZAR1M(ER),TDSEK1M(ER),TDCHF1M(ER),USTHB1F(ER),TDGBP1M(ER),TDAUD1M(ER),BRL$1MF(ER),BLEUR1F(ER),BBCAD1F(ER),CLEUR1F(ER),USHRK1F(ER),TDCZK1M(ER),TDEUR1M(ER),USHUF1F(ER),USINR1F(ER),USIDR1F(ER),USILS1F(ER),TDJPY1M(ER),USMXN1F(ER),TDNZD1M(ER)
1,1998-01-01 00:00:00,7.3605,40.7005,3.58455,nan,1.6929,4.9075,7.92715,1.4565,48.55,1.6451,0.65145,nan,nan,1.4316,nan,nan,34.98549,1.10047,206.79,39.2365,5540,nan,130.05,8.1605,0.57947
2,1998-02-01 00:00:00,7.50685,41.586,3.59035,nan,1.716,4.96625,8.0614,1.46795,51.275,1.63705,0.68607,nan,nan,1.4516,nan,nan,35.14749,1.09155,210.055,38.9255,10275,nan,125.9535,8.5295,0.59018
3,1998-03-01 00:00:00,7.5532,40.043,3.54035,nan,1.6184,4.966,7.97715,1.46027,43.88,1.64514,0.68348,nan,nan,1.4247,nan,nan,34.11249,1.09157,211.075,39.542,8835,nan,124.945,8.59975,0.5863
4,1998-04-01 00:00:00,7.664,38.7155,3.51156,nan,1.6231,5.083,8.0309,1.52837,40.05,1.66875,0.65799,nan,nan,1.4164,nan,nan,34.5085,1.07499,215.88,39.511,8725,nan,133.0775,8.603,0.54765
5,1998-05-01 00:00:00,7.3965,40.6335,3.44785,nan,1.5849,5.087,7.6927,1.48666,39.15,1.66404,0.64888,nan,nan,1.4329,nan,nan,33.008,1.10997,211.875,39.7325,8320,nan,132.8305,8.5985,0.55189
6,1998-06-01 00:00:00,7.50745,39.683,3.56905,nan,1.6781,5.2755,7.80365,1.47142,43.05,1.6374,0.61553,nan,nan,1.4552,nan,nan,33.57199,1.10917,216.22,41.75,11950,nan,139.01,9.0185,0.52693
7,1998-07-01 00:00:00,7.72385,41.704,3.53325,nan,1.654,6.11175,8.05125,1.52383,42.95,1.65777,0.62123,nan,nan,1.4646,nan,nan,33.0365,1.0909,221.995,42.64,15162.5,nan,137.482,9.0615,0.51827
8,1998-08-01 00:00:00,7.61015,42.5515,3.4809,nan,1.7344,6.26875,8.0007,1.49494,41.21,1.6241,0.60258,nan,nan,1.5131,nan,nan,30.98149,1.10504,218.475,42.73,13712.6,nan,144.97,9.0845,0.50801
9,1998-09-01 00:00:00,7.749,43.5825,3.7193,nan,1.7074,6.35275,7.9252,1.43936,40.65,1.6722,0.57986,nan,nan,1.5551,nan,nan,32.12599,1.12705,225.155,42.9913,11650,nan,136.09,10.322,0.50427


# Inverser les valeurs et renommer les colonnes commençant par "US $ TO"

Ce script :
1. Charge un fichier CSV (`filtered_fwd_month.csv`) dans un DataFrame Pandas.
2. Crée une copie du DataFrame d'origine pour effectuer une comparaison après transformation.
3. Identifie toutes les colonnes dont le nom commence par "US $ TO".
4. Applique les transformations suivantes sur ces colonnes :
   - Convertit les valeurs en nombres (`pd.to_numeric`) et remplace les valeurs non numériques par `NaN`.
   - Inverse les valeurs en effectuant `1 / valeur` pour chaque cellule.
5. Renomme les colonnes transformées en remplaçant "US $ TO" par "TO US $" dans leurs noms.
6. Sauvegarde le DataFrame modifié dans un nouveau fichier CSV (`updated_filtered_fwd_month.csv`).
7. Affiche une comparaison entre les colonnes d'origine et les colonnes transformées pour vérifier les changements (limité aux 10 premières lignes).
8. Affiche le chemin du fichier CSV mis à jour.

**Utilisation** :
- Ce script est utilisé pour inverser les valeurs des colonnes de taux de change exprimés en dollars US et pour renommer les colonnes de manière plus intuitive tout en fournissant une preuve des transformations effectuées.


In [21]:
# Import necessary libraries
import pandas as pd

# Load the uploaded CSV file
file_path = 'filtered_fwd_month.csv'
df = pd.read_csv(file_path)

# Make a copy of the original DataFrame for comparison
original_df = df.copy()

# Identify all columns starting with "US $ TO"
columns_to_convert = [col for col in df.columns if col.startswith("US $ TO")]

# Convert the values in the identified columns (1/value)
for col in columns_to_convert:
    df[col] = pd.to_numeric(df[col], errors='coerce')  # Convert to numeric, replace non-numeric with NaN
    df[col] = 1 / df[col]  # Apply the inversion

# Rename the converted columns
renamed_columns = {col: col.replace("US $ TO", "TO US $") for col in columns_to_convert}
df.rename(columns=renamed_columns, inplace=True)

# Save the updated DataFrame back to a CSV file
updated_csv_path = 'updated_filtered_fwd_month.csv'
df.to_csv(updated_csv_path, index=False)

# Show evidence of changes
print("Comparison of Original and Updated DataFrames (Only Affected Columns):")
for col in columns_to_convert:
    new_col_name = renamed_columns[col]
    print(f"\nColumn: {col} -> {new_col_name}")
    comparison = pd.DataFrame({
        "Original": original_df[col],
        "Updated": df[new_col_name]
    }).head(10)  # Show first 10 rows for brevity
    print(comparison)

print(f"\nUpdated CSV saved to: {updated_csv_path}")


Comparison of Original and Updated DataFrames (Only Affected Columns):

Column: US $ TO GBP 1M FWD (LSEG DS) - EXCHANGE RATE -> TO US $ GBP 1M FWD (LSEG DS) - EXCHANGE RATE
      Original   Updated
0  TDGBP1M(ER)       NaN
1       1.6451  0.607866
2      1.63705  0.610855
3      1.64514  0.607851
4      1.66875  0.599251
5      1.66404  0.600947
6       1.6374  0.610724
7      1.65777  0.603220
8       1.6241  0.615726
9       1.6722  0.598015

Column: US $ TO AUD 1M FWD (LSEG DS) - EXCHANGE RATE -> TO US $ AUD 1M FWD (LSEG DS) - EXCHANGE RATE
      Original   Updated
0  TDAUD1M(ER)       NaN
1      0.65145  1.535037
2      0.68607  1.457577
3      0.68348  1.463101
4      0.65799  1.519780
5      0.64888  1.541117
6      0.61553  1.624616
7      0.62123  1.609710
8      0.60258  1.659531
9      0.57986  1.724554

Column: US $ TO EURO 1M FWD (LSEG DS) - EXCHANGE RATE -> TO US $ EURO 1M FWD (LSEG DS) - EXCHANGE RATE
      Original   Updated
0  TDEUR1M(ER)       NaN
1      1.10047  0.908

# Afficher un DataFrame mis à jour avec styles et analyses

Ce script :
1. Charge un fichier CSV mis à jour (`updated_filtered_fwd_month.csv`) dans un DataFrame Pandas.
2. Affiche le DataFrame entier dans une table stylisée avec :
   - Un titre descriptif : "Updated DataFrame: Taux de Change Forward et Taux de Devise".
3. Analyse et affiche les valeurs manquantes (N/A) par colonne dans une table stylisée intitulée "Missing Values".
4. Génère un résumé statistique des colonnes numériques (moyenne, min, max, etc.) dans une table stylisée intitulée "Summary Statistics for Taux de Change".
5. Affiche un aperçu des 10 premières lignes du DataFrame avec un titre : "Preview of Data: First 10 Rows".

**Utilisation** :
- Ce script est conçu pour explorer et valider un DataFrame mis à jour avec une présentation améliorée des données, des valeurs manquantes et des statistiques clés.


In [22]:
# Import necessary libraries
import pandas as pd
from IPython.display import display

# Load the updated CSV file
updated_csv_path = 'updated_filtered_fwd_month.csv'  # Replace with your updated file
df = pd.read_csv(updated_csv_path)

# Step 1: Display the DataFrame in a styled table
print("Displaying the updated DataFrame:")
display(df.style.set_caption("Updated DataFrame: Taux de Change Forward et Taux de Devise")
        .set_table_styles([{'selector': 'caption', 'props': [('font-size', '16px'), ('text-align', 'left')]}]))

# Step 2: Check for missing values
print("\nMissing Values Per Column:")
missing_values = pd.DataFrame(df.isnull().sum(), columns=["Missing Values"])
display(missing_values.style.set_caption("Missing Values")
        .set_table_styles([{'selector': 'caption', 'props': [('font-size', '16px'), ('text-align', 'left')]}]))

# Step 3: Summary statistics for numerical columns
print("\nSummary Statistics (Numerical Columns):")
display(df.describe().style.set_caption("Summary Statistics for Taux de Change")
        .set_table_styles([{'selector': 'caption', 'props': [('font-size', '16px'), ('text-align', 'left')]}]))

# Step 4: Display the first 10 rows for a preview of data
print("\nPreview of the DataFrame:")
display(df.head(10).style.set_caption("Preview of Data: First 10 Rows")
        .set_table_styles([{'selector': 'caption', 'props': [('font-size', '16px'), ('text-align', 'left')]}]))


Displaying the updated DataFrame:


,Unnamed: 0,NOK TO US $ 1M FWD (LSEG DS) - EXCHANGE RATE,PHP TO USD 1M FWD OR (WMR) - EXCHANGE RATE,PLN TO US $ 1M FWD (LSEG DS) - EXCHANGE RATE,RUB TO USD 1M FWD OR (WMR) - EXCHANGE RATE,SINGAPORE $ TO US $ 1M FWD (BBI) - EXCHANGE RATE,ZAR TO USD 1M FWD (LSEG DS) - EXCHANGE RATE,SEK TO US $ 1M FWD (LSEG DS) - EXCHANGE RATE,SWISS FRANC TO US $ 1M FWD (LSEG DS) - EXCHANGE RATE,THB TO USD 1M FWD OR (WMR) - EXCHANGE RATE,TO US $ GBP 1M FWD (LSEG DS) - EXCHANGE RATE,TO US $ AUD 1M FWD (LSEG DS) - EXCHANGE RATE,BRL TO USD 1M FWD OFF (LSEG DS) - EXCHANGE RATE,BGN TO EUR 1M FWD OR (WMR) - EXCHANGE RATE,CANADIAN $ TO US $ 1M FWD (BBI) - EXCHANGE RATE,CLP TO EUR 1M FWD OR (WMR) - EXCHANGE RATE,HRK TO USD 1M FWD OR (WMR) - EXCHANGE RATE,CZK TO US $ 1M FWD (LSEG DS) - EXCHANGE RATE,TO US $ EURO 1M FWD (LSEG DS) - EXCHANGE RATE,HUF TO USD 1M FWD OR (WMR) - EXCHANGE RATE,INR TO USD 1M FWD OR (WMR) - EXCHANGE RATE,IDR TO USD 1M FWD OR (WMR) - EXCHANGE RATE,ILS TO USD 1M FWD OR (WMR) - EXCHANGE RATE,JPY TO US $ 1M FWD (LSEG DS) - EXCHANGE RATE,MXN TO USD 1M FWD OR (WMR) - EXCHANGE RATE,TO US $ NZD 1M FWD (LSEG DS) - EXCHANGE RATE
0,Code,TDNOK1M(ER),USPHP1F(ER),TDPLN1M(ER),USRUB1F(ER),BBSGD1F(ER),TDZAR1M(ER),TDSEK1M(ER),TDCHF1M(ER),USTHB1F(ER),nan,nan,BRL$1MF(ER),BLEUR1F(ER),BBCAD1F(ER),CLEUR1F(ER),USHRK1F(ER),TDCZK1M(ER),nan,USHUF1F(ER),USINR1F(ER),USIDR1F(ER),USILS1F(ER),TDJPY1M(ER),USMXN1F(ER),nan
1,1998-01-01 00:00:00,7.3605,40.7005,3.58455,nan,1.6929,4.9075,7.92715,1.4565,48.55,0.607866,1.535037,nan,nan,1.4316,nan,nan,34.98549,0.908703,206.79,39.2365,5540,nan,130.05,8.1605,1.725715
2,1998-02-01 00:00:00,7.50685,41.586,3.59035,nan,1.716,4.96625,8.0614,1.46795,51.275,0.610855,1.457577,nan,nan,1.4516,nan,nan,35.14749,0.916128,210.055,38.9255,10275,nan,125.9535,8.5295,1.694398
3,1998-03-01 00:00:00,7.5532,40.043,3.54035,nan,1.6184,4.966,7.97715,1.46027,43.88,0.607851,1.463101,nan,nan,1.4247,nan,nan,34.11249,0.916112,211.075,39.542,8835,nan,124.945,8.59975,1.705611
4,1998-04-01 00:00:00,7.664,38.7155,3.51156,nan,1.6231,5.083,8.0309,1.52837,40.05,0.599251,1.519780,nan,nan,1.4164,nan,nan,34.5085,0.930241,215.88,39.511,8725,nan,133.0775,8.603,1.825984
5,1998-05-01 00:00:00,7.3965,40.6335,3.44785,nan,1.5849,5.087,7.6927,1.48666,39.15,0.600947,1.541117,nan,nan,1.4329,nan,nan,33.008,0.900925,211.875,39.7325,8320,nan,132.8305,8.5985,1.811955
6,1998-06-01 00:00:00,7.50745,39.683,3.56905,nan,1.6781,5.2755,7.80365,1.47142,43.05,0.610724,1.624616,nan,nan,1.4552,nan,nan,33.57199,0.901575,216.22,41.75,11950,nan,139.01,9.0185,1.897785
7,1998-07-01 00:00:00,7.72385,41.704,3.53325,nan,1.654,6.11175,8.05125,1.52383,42.95,0.603220,1.609710,nan,nan,1.4646,nan,nan,33.0365,0.916674,221.995,42.64,15162.5,nan,137.482,9.0615,1.929496
8,1998-08-01 00:00:00,7.61015,42.5515,3.4809,nan,1.7344,6.26875,8.0007,1.49494,41.21,0.615726,1.659531,nan,nan,1.5131,nan,nan,30.98149,0.904945,218.475,42.73,13712.6,nan,144.97,9.0845,1.968465
9,1998-09-01 00:00:00,7.749,43.5825,3.7193,nan,1.7074,6.35275,7.9252,1.43936,40.65,0.598015,1.724554,nan,nan,1.5551,nan,nan,32.12599,0.887272,225.155,42.9913,11650,nan,136.09,10.322,1.983065



Missing Values Per Column:


,Missing Values
Unnamed: 0,1
NOK TO US $ 1M FWD (LSEG DS) - EXCHANGE RATE,1
PHP TO USD 1M FWD OR (WMR) - EXCHANGE RATE,1
PLN TO US $ 1M FWD (LSEG DS) - EXCHANGE RATE,1
RUB TO USD 1M FWD OR (WMR) - EXCHANGE RATE,76
SINGAPORE $ TO US $ 1M FWD (BBI) - EXCHANGE RATE,1
ZAR TO USD 1M FWD (LSEG DS) - EXCHANGE RATE,1
SEK TO US $ 1M FWD (LSEG DS) - EXCHANGE RATE,1
SWISS FRANC TO US $ 1M FWD (LSEG DS) - EXCHANGE RATE,1
THB TO USD 1M FWD OR (WMR) - EXCHANGE RATE,1



Summary Statistics (Numerical Columns):


,TO US $ GBP 1M FWD (LSEG DS) - EXCHANGE RATE,TO US $ AUD 1M FWD (LSEG DS) - EXCHANGE RATE,TO US $ EURO 1M FWD (LSEG DS) - EXCHANGE RATE,TO US $ NZD 1M FWD (LSEG DS) - EXCHANGE RATE
count,322.000000,322.000000,322.000000,322.000000
mean,0.662938,1.372136,0.858917,1.573760
std,0.091560,0.247331,0.114786,0.309851
min,0.481348,0.915508,0.634288,1.142870
25%,0.605201,1.213566,0.766583,1.378564
50%,0.646567,1.356917,0.855140,1.487332
75%,0.744020,1.516910,0.917290,1.686180
max,0.882870,2.086594,1.180233,2.521496



Preview of the DataFrame:


,Unnamed: 0,NOK TO US $ 1M FWD (LSEG DS) - EXCHANGE RATE,PHP TO USD 1M FWD OR (WMR) - EXCHANGE RATE,PLN TO US $ 1M FWD (LSEG DS) - EXCHANGE RATE,RUB TO USD 1M FWD OR (WMR) - EXCHANGE RATE,SINGAPORE $ TO US $ 1M FWD (BBI) - EXCHANGE RATE,ZAR TO USD 1M FWD (LSEG DS) - EXCHANGE RATE,SEK TO US $ 1M FWD (LSEG DS) - EXCHANGE RATE,SWISS FRANC TO US $ 1M FWD (LSEG DS) - EXCHANGE RATE,THB TO USD 1M FWD OR (WMR) - EXCHANGE RATE,TO US $ GBP 1M FWD (LSEG DS) - EXCHANGE RATE,TO US $ AUD 1M FWD (LSEG DS) - EXCHANGE RATE,BRL TO USD 1M FWD OFF (LSEG DS) - EXCHANGE RATE,BGN TO EUR 1M FWD OR (WMR) - EXCHANGE RATE,CANADIAN $ TO US $ 1M FWD (BBI) - EXCHANGE RATE,CLP TO EUR 1M FWD OR (WMR) - EXCHANGE RATE,HRK TO USD 1M FWD OR (WMR) - EXCHANGE RATE,CZK TO US $ 1M FWD (LSEG DS) - EXCHANGE RATE,TO US $ EURO 1M FWD (LSEG DS) - EXCHANGE RATE,HUF TO USD 1M FWD OR (WMR) - EXCHANGE RATE,INR TO USD 1M FWD OR (WMR) - EXCHANGE RATE,IDR TO USD 1M FWD OR (WMR) - EXCHANGE RATE,ILS TO USD 1M FWD OR (WMR) - EXCHANGE RATE,JPY TO US $ 1M FWD (LSEG DS) - EXCHANGE RATE,MXN TO USD 1M FWD OR (WMR) - EXCHANGE RATE,TO US $ NZD 1M FWD (LSEG DS) - EXCHANGE RATE
0,Code,TDNOK1M(ER),USPHP1F(ER),TDPLN1M(ER),USRUB1F(ER),BBSGD1F(ER),TDZAR1M(ER),TDSEK1M(ER),TDCHF1M(ER),USTHB1F(ER),nan,nan,BRL$1MF(ER),BLEUR1F(ER),BBCAD1F(ER),CLEUR1F(ER),USHRK1F(ER),TDCZK1M(ER),nan,USHUF1F(ER),USINR1F(ER),USIDR1F(ER),USILS1F(ER),TDJPY1M(ER),USMXN1F(ER),nan
1,1998-01-01 00:00:00,7.3605,40.7005,3.58455,nan,1.6929,4.9075,7.92715,1.4565,48.55,0.607866,1.535037,nan,nan,1.4316,nan,nan,34.98549,0.908703,206.79,39.2365,5540,nan,130.05,8.1605,1.725715
2,1998-02-01 00:00:00,7.50685,41.586,3.59035,nan,1.716,4.96625,8.0614,1.46795,51.275,0.610855,1.457577,nan,nan,1.4516,nan,nan,35.14749,0.916128,210.055,38.9255,10275,nan,125.9535,8.5295,1.694398
3,1998-03-01 00:00:00,7.5532,40.043,3.54035,nan,1.6184,4.966,7.97715,1.46027,43.88,0.607851,1.463101,nan,nan,1.4247,nan,nan,34.11249,0.916112,211.075,39.542,8835,nan,124.945,8.59975,1.705611
4,1998-04-01 00:00:00,7.664,38.7155,3.51156,nan,1.6231,5.083,8.0309,1.52837,40.05,0.599251,1.519780,nan,nan,1.4164,nan,nan,34.5085,0.930241,215.88,39.511,8725,nan,133.0775,8.603,1.825984
5,1998-05-01 00:00:00,7.3965,40.6335,3.44785,nan,1.5849,5.087,7.6927,1.48666,39.15,0.600947,1.541117,nan,nan,1.4329,nan,nan,33.008,0.900925,211.875,39.7325,8320,nan,132.8305,8.5985,1.811955
6,1998-06-01 00:00:00,7.50745,39.683,3.56905,nan,1.6781,5.2755,7.80365,1.47142,43.05,0.610724,1.624616,nan,nan,1.4552,nan,nan,33.57199,0.901575,216.22,41.75,11950,nan,139.01,9.0185,1.897785
7,1998-07-01 00:00:00,7.72385,41.704,3.53325,nan,1.654,6.11175,8.05125,1.52383,42.95,0.603220,1.609710,nan,nan,1.4646,nan,nan,33.0365,0.916674,221.995,42.64,15162.5,nan,137.482,9.0615,1.929496
8,1998-08-01 00:00:00,7.61015,42.5515,3.4809,nan,1.7344,6.26875,8.0007,1.49494,41.21,0.615726,1.659531,nan,nan,1.5131,nan,nan,30.98149,0.904945,218.475,42.73,13712.6,nan,144.97,9.0845,1.968465
9,1998-09-01 00:00:00,7.749,43.5825,3.7193,nan,1.7074,6.35275,7.9252,1.43936,40.65,0.598015,1.724554,nan,nan,1.5551,nan,nan,32.12599,0.887272,225.155,42.9913,11650,nan,136.09,10.322,1.983065


# Transformer les colonnes contenant "TO EUR" en "TO USD" en utilisant une colonne de conversion

Ce script :
1. Charge un fichier CSV (`updated_filtered_fwd_month.csv`) dans un DataFrame Pandas.
2. Identifie une colonne spécifique pour la conversion : `"TO US $ EURO 1M FWD (LSEG DS) - EXCHANGE RATE"`.
   - Si cette colonne est absente, le script génère une erreur.
3. Identifie toutes les colonnes contenant "TO EUR" dans leur nom.
4. Multiplie chaque colonne "TO EUR" par la colonne de conversion pour obtenir les valeurs correspondantes en "TO USD".
5. Sauvegarde le DataFrame transformé dans un nouveau fichier CSV : `transformed_to_usd_filtered_fwd_month.csv`.
6. Fournit des preuves des transformations en affichant une comparaison entre les colonnes "TO EUR" d'origine (recalculées) et les colonnes "TO USD" transformées (premières 10 lignes).
7. Affiche le chemin du fichier CSV mis à jour.

**Utilisation** :
- Ce script est utile pour convertir des colonnes de taux de change exprimées en EUR vers des valeurs en USD, en utilisant une colonne de conversion définie.


In [23]:
# Import necessary libraries
import pandas as pd

# Load the CSV file
file_path = 'updated_filtered_fwd_month.csv'
df = pd.read_csv(file_path)

# Identify the column for conversion to USD: "TO US $ EURO 1M FWD (LSEG DS) - EXCHANGE RATE"
conversion_column = "TO US $ EURO 1M FWD (LSEG DS) - EXCHANGE RATE"

if conversion_column not in df.columns:
    raise ValueError(f"The required conversion column '{conversion_column}' is not in the DataFrame.")

# Identify all columns containing "TO EUR"
columns_to_transform = [col for col in df.columns if "TO EUR" in col]

# Multiply each "TO EUR" column by the conversion column
for col in columns_to_transform:
    df[col] = pd.to_numeric(df[col], errors='coerce')  # Convert to numeric, replace non-numeric with NaN
    df[col] = df[col] * pd.to_numeric(df[conversion_column], errors='coerce')  # Multiply by the conversion column

# Save the updated DataFrame back to a CSV file
updated_csv_path = 'transformed_to_usd_filtered_fwd_month.csv'
df.to_csv(updated_csv_path, index=False)

# Show evidence of the transformation
print("Transformation of 'TO EUR' columns to USD (Only Affected Columns):")
for col in columns_to_transform:
    print(f"\nColumn: {col} (Transformed to USD)")
    comparison = pd.DataFrame({
        "Original (TO EUR)": df[col] / df[conversion_column],  # Reverse the transformation for comparison
        "Converted (TO USD)": df[col]
    }).head(10)  # Show first 10 rows for brevity
    print(comparison)

print(f"\nUpdated CSV saved to: {updated_csv_path}")


Transformation of 'TO EUR' columns to USD (Only Affected Columns):

Column: BGN TO EUR 1M FWD OR (WMR) - EXCHANGE RATE (Transformed to USD)
   Original (TO EUR)  Converted (TO USD)
0                NaN                 NaN
1                NaN                 NaN
2                NaN                 NaN
3                NaN                 NaN
4                NaN                 NaN
5                NaN                 NaN
6                NaN                 NaN
7                NaN                 NaN
8                NaN                 NaN
9                NaN                 NaN

Column: CLP TO EUR 1M FWD OR (WMR) - EXCHANGE RATE (Transformed to USD)
   Original (TO EUR)  Converted (TO USD)
0                NaN                 NaN
1                NaN                 NaN
2                NaN                 NaN
3                NaN                 NaN
4                NaN                 NaN
5                NaN                 NaN
6                NaN                 NaN
7       

# Afficher un DataFrame transformé (colonnes "TO EUR" converties en "TO USD") avec styles

Ce script :
1. Charge un fichier CSV transformé (`transformed_to_usd_filtered_fwd_month.csv`) dans un DataFrame Pandas.
2. Affiche les 10 dernières lignes du DataFrame dans une table stylisée avec :
   - Un titre descriptif : "Transformed DataFrame: TO EUR Converted to USD (Last Rows)".
   - Un dégradé de couleurs (`coolwarm`) pour faciliter la lecture des variations de valeurs.
3. Analyse et affiche les valeurs manquantes (N/A) dans le DataFrame sous forme de table stylisée avec un titre : "Missing Values Per Column".

**Utilisation** :
- Ce script est conçu pour inspecter et valider un DataFrame après la conversion de colonnes "TO EUR" en "TO USD", tout en offrant une présentation améliorée des données et des valeurs manquantes.


In [25]:
# Import necessary libraries
import pandas as pd
from IPython.display import display

# Load the transformed CSV file
transformed_csv_path = 'transformed_to_usd_filtered_fwd_month.csv'
df = pd.read_csv(transformed_csv_path)

# Step 1: Display the DataFrame in a styled table
print("Displaying the Transformed DataFrame (Last Rows):")
display(df.tail(10).style.set_caption("Transformed DataFrame: TO EUR Converted to USD (Last Rows)")
        .set_table_styles([{'selector': 'caption', 'props': [('font-size', '16px'), ('text-align', 'left')]}])
        .background_gradient(cmap="coolwarm", axis=None))  # Apply gradient styling

# Step 2: Summary of missing values for context
print("\nMissing Values in Transformed DataFrame:")
missing_values = pd.DataFrame(df.isnull().sum(), columns=["Missing Values"])
display(missing_values.style.set_caption("Missing Values Per Column")
        .set_table_styles([{'selector': 'caption', 'props': [('font-size', '16px'), ('text-align', 'left')]}]))


Displaying the Transformed DataFrame (Last Rows):


,Unnamed: 0,NOK TO US $ 1M FWD (LSEG DS) - EXCHANGE RATE,PHP TO USD 1M FWD OR (WMR) - EXCHANGE RATE,PLN TO US $ 1M FWD (LSEG DS) - EXCHANGE RATE,RUB TO USD 1M FWD OR (WMR) - EXCHANGE RATE,SINGAPORE $ TO US $ 1M FWD (BBI) - EXCHANGE RATE,ZAR TO USD 1M FWD (LSEG DS) - EXCHANGE RATE,SEK TO US $ 1M FWD (LSEG DS) - EXCHANGE RATE,SWISS FRANC TO US $ 1M FWD (LSEG DS) - EXCHANGE RATE,THB TO USD 1M FWD OR (WMR) - EXCHANGE RATE,TO US $ GBP 1M FWD (LSEG DS) - EXCHANGE RATE,TO US $ AUD 1M FWD (LSEG DS) - EXCHANGE RATE,BRL TO USD 1M FWD OFF (LSEG DS) - EXCHANGE RATE,BGN TO EUR 1M FWD OR (WMR) - EXCHANGE RATE,CANADIAN $ TO US $ 1M FWD (BBI) - EXCHANGE RATE,CLP TO EUR 1M FWD OR (WMR) - EXCHANGE RATE,HRK TO USD 1M FWD OR (WMR) - EXCHANGE RATE,CZK TO US $ 1M FWD (LSEG DS) - EXCHANGE RATE,TO US $ EURO 1M FWD (LSEG DS) - EXCHANGE RATE,HUF TO USD 1M FWD OR (WMR) - EXCHANGE RATE,INR TO USD 1M FWD OR (WMR) - EXCHANGE RATE,IDR TO USD 1M FWD OR (WMR) - EXCHANGE RATE,ILS TO USD 1M FWD OR (WMR) - EXCHANGE RATE,JPY TO US $ 1M FWD (LSEG DS) - EXCHANGE RATE,MXN TO USD 1M FWD OR (WMR) - EXCHANGE RATE,TO US $ NZD 1M FWD (LSEG DS) - EXCHANGE RATE
314,2024-02-01 00:00:00,10.43555,56.155,3.97157,91.35925,1.3351,18.65355,10.38008,0.855,35.3788,0.784369,1.519711,4.9291,1.796784,1.3377,931.125110,6.94946,22.88016,0.918780,354.4586,83.0607,15772.5,3.65752,145.7652,17.201,1.628479
315,2024-03-01 00:00:00,10.51242,56.0495,3.98405,92.70025,1.3428,19.15897,10.30715,0.88058,35.859,0.789983,1.530152,4.9754,1.802780,1.3548,969.234238,6.94466,23.4185,0.921761,363.9866,82.9913,15714,3.56275,149.4041,17.08955,1.637304
316,2024-04-01 00:00:00,10.94689,56.2825,3.98409,93.3209,1.3511,19.00584,10.78385,0.90111,36.34,0.796578,1.539598,5.0701,1.818481,1.3571,987.037769,7.00399,23.55125,0.929817,367.6944,83.47,15904.25,3.67377,150.8808,16.72045,1.679318
317,2024-05-01 00:00:00,11.02191,57.812,4.03779,94.7151,1.3624,18.6266,10.91746,0.91281,36.9675,0.798365,1.532191,5.21825,1.823026,1.376,957.752188,7.04378,23.41879,0.932079,365.4092,83.5238,16271.75,3.7509,154.1118,17.1171,1.687479
318,2024-06-01 00:00:00,10.4474,58.728,3.92445,90.4527,1.3452,18.55923,10.39624,0.89294,36.7005,0.780762,1.493987,5.2641,1.789761,1.3634,907.271843,6.92225,22.62175,0.916036,360.3112,83.2188,16237,3.6635,155.3643,17.63115,1.615248
319,2024-07-01 00:00:00,10.65005,58.645,4.02077,87.8385,1.3557,18.41624,10.60091,0.89938,36.6145,0.790426,1.500780,5.6806,1.816593,1.3727,940.186540,7.01148,23.39185,0.929912,368.2215,83.5263,16330,3.75788,160.7209,18.51235,1.645278
320,2024-08-01 00:00:00,10.94662,58.37,3.98551,86.3591,1.335,18.32208,10.71435,0.86993,35.5175,0.784628,1.537114,5.7676,1.809641,1.3843,936.919597,6.97115,23.4389,0.925241,367.3927,83.8088,16251,3.7849,148.6684,18.68115,1.682284
321,2024-09-01 00:00:00,10.58181,56.387,3.8583,90.3832,1.3049,17.8775,10.22708,0.84875,34.14,0.760364,1.471259,5.63915,1.764945,1.3481,914.862459,6.79723,22.59364,0.901908,354.7097,84.0013,15543.5,3.64729,146.2278,19.87775,1.604621
322,2024-10-01 00:00:00,10.60221,56.164,3.87428,95.2304,1.2868,17.45786,10.2577,0.84379,32.5185,0.752740,1.452032,5.4568,1.764910,1.3491,900.759072,6.79303,22.82761,0.902250,360.0547,83.9363,15219,3.7536,142.9332,19.8307,1.591571
323,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan



Missing Values in Transformed DataFrame:


,Missing Values
Unnamed: 0,1
NOK TO US $ 1M FWD (LSEG DS) - EXCHANGE RATE,1
PHP TO USD 1M FWD OR (WMR) - EXCHANGE RATE,1
PLN TO US $ 1M FWD (LSEG DS) - EXCHANGE RATE,1
RUB TO USD 1M FWD OR (WMR) - EXCHANGE RATE,76
SINGAPORE $ TO US $ 1M FWD (BBI) - EXCHANGE RATE,1
ZAR TO USD 1M FWD (LSEG DS) - EXCHANGE RATE,1
SEK TO US $ 1M FWD (LSEG DS) - EXCHANGE RATE,1
SWISS FRANC TO US $ 1M FWD (LSEG DS) - EXCHANGE RATE,1
THB TO USD 1M FWD OR (WMR) - EXCHANGE RATE,1


# Renommer les colonnes contenant "TO EUR" en "TO USD"

Ce script :
1. Charge un fichier CSV transformé (`transformed_to_usd_filtered_fwd_month.csv`) dans un DataFrame Pandas.
2. Identifie toutes les colonnes contenant "TO EUR" dans leur nom.
3. Renomme les colonnes identifiées en remplaçant "TO EUR" par "TO USD".
4. Sauvegarde le DataFrame mis à jour dans un nouveau fichier CSV : `renamed_to_usd_filtered_fwd_month.csv`.
5. Fournit des preuves du renommage en affichant les noms des colonnes avant et après transformation.
6. Affiche les 10 dernières lignes du DataFrame mis à jour dans une table stylisée avec un titre descriptif : "Transformed DataFrame: Columns Renamed to TO USD".

**Utilisation** :
- Ce script est utilisé pour uniformiser les noms de colonnes en remplaçant "TO EUR" par "TO USD", notamment dans les jeux de données liés aux taux de change.


In [26]:
# Import necessary libraries
import pandas as pd

# Load the transformed CSV file
transformed_csv_path = 'transformed_to_usd_filtered_fwd_month.csv'
df = pd.read_csv(transformed_csv_path)

# Identify all columns containing "TO EUR"
columns_to_rename = [col for col in df.columns if "TO EUR" in col]

# Rename the columns by replacing "TO EUR" with "TO USD"
renamed_columns = {col: col.replace("TO EUR", "TO USD") for col in columns_to_rename}
df.rename(columns=renamed_columns, inplace=True)

# Save the updated DataFrame back to a CSV file
updated_csv_path = 'renamed_to_usd_filtered_fwd_month.csv'
df.to_csv(updated_csv_path, index=False)

# Show evidence of renaming
print("Renamed Columns:")
for old_col, new_col in renamed_columns.items():
    print(f"'{old_col}' renamed to '{new_col}'")

# Optional: Display the updated DataFrame (last rows for inspection)
print("\nDisplaying the Transformed DataFrame (Last Rows):")
from IPython.display import display
display(df.tail(10).style.set_caption("Transformed DataFrame: Columns Renamed to TO USD")
        .set_table_styles([{'selector': 'caption', 'props': [('font-size', '16px'), ('text-align', 'left')]}]))


Renamed Columns:
'BGN TO EUR 1M FWD OR (WMR) - EXCHANGE RATE' renamed to 'BGN TO USD 1M FWD OR (WMR) - EXCHANGE RATE'
'CLP TO EUR 1M FWD OR (WMR) - EXCHANGE RATE' renamed to 'CLP TO USD 1M FWD OR (WMR) - EXCHANGE RATE'

Displaying the Transformed DataFrame (Last Rows):


,Unnamed: 0,NOK TO US $ 1M FWD (LSEG DS) - EXCHANGE RATE,PHP TO USD 1M FWD OR (WMR) - EXCHANGE RATE,PLN TO US $ 1M FWD (LSEG DS) - EXCHANGE RATE,RUB TO USD 1M FWD OR (WMR) - EXCHANGE RATE,SINGAPORE $ TO US $ 1M FWD (BBI) - EXCHANGE RATE,ZAR TO USD 1M FWD (LSEG DS) - EXCHANGE RATE,SEK TO US $ 1M FWD (LSEG DS) - EXCHANGE RATE,SWISS FRANC TO US $ 1M FWD (LSEG DS) - EXCHANGE RATE,THB TO USD 1M FWD OR (WMR) - EXCHANGE RATE,TO US $ GBP 1M FWD (LSEG DS) - EXCHANGE RATE,TO US $ AUD 1M FWD (LSEG DS) - EXCHANGE RATE,BRL TO USD 1M FWD OFF (LSEG DS) - EXCHANGE RATE,BGN TO USD 1M FWD OR (WMR) - EXCHANGE RATE,CANADIAN $ TO US $ 1M FWD (BBI) - EXCHANGE RATE,CLP TO USD 1M FWD OR (WMR) - EXCHANGE RATE,HRK TO USD 1M FWD OR (WMR) - EXCHANGE RATE,CZK TO US $ 1M FWD (LSEG DS) - EXCHANGE RATE,TO US $ EURO 1M FWD (LSEG DS) - EXCHANGE RATE,HUF TO USD 1M FWD OR (WMR) - EXCHANGE RATE,INR TO USD 1M FWD OR (WMR) - EXCHANGE RATE,IDR TO USD 1M FWD OR (WMR) - EXCHANGE RATE,ILS TO USD 1M FWD OR (WMR) - EXCHANGE RATE,JPY TO US $ 1M FWD (LSEG DS) - EXCHANGE RATE,MXN TO USD 1M FWD OR (WMR) - EXCHANGE RATE,TO US $ NZD 1M FWD (LSEG DS) - EXCHANGE RATE
314,2024-02-01 00:00:00,10.43555,56.155,3.97157,91.35925,1.3351,18.65355,10.38008,0.855,35.3788,0.784369,1.519711,4.9291,1.796784,1.3377,931.125110,6.94946,22.88016,0.918780,354.4586,83.0607,15772.5,3.65752,145.7652,17.201,1.628479
315,2024-03-01 00:00:00,10.51242,56.0495,3.98405,92.70025,1.3428,19.15897,10.30715,0.88058,35.859,0.789983,1.530152,4.9754,1.802780,1.3548,969.234238,6.94466,23.4185,0.921761,363.9866,82.9913,15714,3.56275,149.4041,17.08955,1.637304
316,2024-04-01 00:00:00,10.94689,56.2825,3.98409,93.3209,1.3511,19.00584,10.78385,0.90111,36.34,0.796578,1.539598,5.0701,1.818481,1.3571,987.037769,7.00399,23.55125,0.929817,367.6944,83.47,15904.25,3.67377,150.8808,16.72045,1.679318
317,2024-05-01 00:00:00,11.02191,57.812,4.03779,94.7151,1.3624,18.6266,10.91746,0.91281,36.9675,0.798365,1.532191,5.21825,1.823026,1.376,957.752188,7.04378,23.41879,0.932079,365.4092,83.5238,16271.75,3.7509,154.1118,17.1171,1.687479
318,2024-06-01 00:00:00,10.4474,58.728,3.92445,90.4527,1.3452,18.55923,10.39624,0.89294,36.7005,0.780762,1.493987,5.2641,1.789761,1.3634,907.271843,6.92225,22.62175,0.916036,360.3112,83.2188,16237,3.6635,155.3643,17.63115,1.615248
319,2024-07-01 00:00:00,10.65005,58.645,4.02077,87.8385,1.3557,18.41624,10.60091,0.89938,36.6145,0.790426,1.500780,5.6806,1.816593,1.3727,940.186540,7.01148,23.39185,0.929912,368.2215,83.5263,16330,3.75788,160.7209,18.51235,1.645278
320,2024-08-01 00:00:00,10.94662,58.37,3.98551,86.3591,1.335,18.32208,10.71435,0.86993,35.5175,0.784628,1.537114,5.7676,1.809641,1.3843,936.919597,6.97115,23.4389,0.925241,367.3927,83.8088,16251,3.7849,148.6684,18.68115,1.682284
321,2024-09-01 00:00:00,10.58181,56.387,3.8583,90.3832,1.3049,17.8775,10.22708,0.84875,34.14,0.760364,1.471259,5.63915,1.764945,1.3481,914.862459,6.79723,22.59364,0.901908,354.7097,84.0013,15543.5,3.64729,146.2278,19.87775,1.604621
322,2024-10-01 00:00:00,10.60221,56.164,3.87428,95.2304,1.2868,17.45786,10.2577,0.84379,32.5185,0.752740,1.452032,5.4568,1.764910,1.3491,900.759072,6.79303,22.82761,0.902250,360.0547,83.9363,15219,3.7536,142.9332,19.8307,1.591571
323,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan
